# Дополнительное домашнее задание №3*

Это просто вторая часть из [лабараторной работы №3](https://github.com/ml-mipt/ml-mipt/blob/b9f61d59cadbe825f476befa954d75ebbb30c30e/homeworks/Lab3_DL/Lab3_DL.ipynb) с курса ФИВТа

Решенный ноутбук нужно загрузить в [форму](http://bit.ly/dafe_hw)  
Дедлайн: __24.05.2019__

В этой части задания мы научимся генерировать текст с помощью нейронных сетей. Конкретнее, обучим нейронную сеть на сонетах Шекспира и попросим нейросеть написать свой сонет.

Генерация текста обычно включает в себя следующие шаги:
    
1. Загрузка данных.
2. Создание словарей слов/символов.
3. Препроцессинг данных.
4. Обучение модели (нейросети).
5. Генерация нового текста.


## Предобработка данных

Для начала загрузим данные. Файл с сонетами Шекспира доступен по [ссылке](http://www.gutenberg.org/ebooks/1041?msg=welcome_stranger). Кроме того, он находится рядом с этим ноутбуком (`sonnetes.txt`).

Базовая предобработка уже сделана: текст состоит непосредственно из поэм Шекспира и названий/номеров глав, все техническая информация удалена.

In [1]:
with open('sonnets.txt', 'r') as iofile:
    text = iofile.readlines()
    
TEXT_START = 45
TEXT_END = -368
text = text[TEXT_START:TEXT_END]
assert len(text) == 2616

Так как в этот раз мы хотим научиться предсказывать текст, понизим сложность задачи и приведем текст к нижнему регистру.

В настоящий момент переменная `text` представляет собой список из строк. Объедините все строки в одну и приведите к нижнему регистру. 

In [81]:
# Объедините все строки в одну и приведите к нижнему регистру.
# Результат запишите в переменную text.
import string
import torch.cuda as cuda
# Your great code here
text = [line.lower() for line in text]
text = ''.join(text)

assert len(text) == 100223, 'Are you sure you have concatenated all the strings?'
assert not any([x in set(text) for x in string.ascii_uppercase]), 'Uppercase letters are present'
print('Отлично!')

Отлично!


Выделите множество всех символов, с которыми нам довелось встретиться в переменную `tokens`.

In [3]:
import numpy as np
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

words = list(set(tokenizer.tokenize(text)))
tokens = np.array(list(set(text)))

In [286]:
# print(words)

Постройте словарь `token_to_idx` вида <символ>: <индекс> и словарь `idx_to_token` вида <индекс>: <символ>.

In [4]:
import itertools
from collections import Counter
def build_vocab(docs, min_count=None):
    docs = itertools.chain.from_iterable(docs)
    c = Counter(docs)
    # count all tokens in all documents and filter those that appear less than min_count
    if min_count is not None:
        for word in c:
            if c[word] < min_count:
                c[word] = -1
        c += Counter()
    num_tokens = len(list(c))
    
    # create the vocabulary mapping
    vocab = dict(zip(sorted(list(c)), range(num_tokens)))

    return vocab
# словарь вида <индекс>:<символ>
# Your great code here
token_to_idx = build_vocab(text)
print(token_to_idx)
# словарь вида <символ>:<индекс>
# Your great code here
idx_to_token = dict((v,k) for k,v in token_to_idx.items())

{'\n': 0, ' ': 1, '!': 2, "'": 3, '(': 4, ')': 5, ',': 6, '-': 7, '.': 8, ':': 9, ';': 10, '?': 11, 'a': 12, 'b': 13, 'c': 14, 'd': 15, 'e': 16, 'f': 17, 'g': 18, 'h': 19, 'i': 20, 'j': 21, 'k': 22, 'l': 23, 'm': 24, 'n': 25, 'o': 26, 'p': 27, 'q': 28, 'r': 29, 's': 30, 't': 31, 'u': 32, 'v': 33, 'w': 34, 'x': 35, 'y': 36, 'z': 37}


In [5]:
num_tokens = len(tokens)

In [6]:
# print(enc.transform(dictlist))
def to_matrix(names, max_len=None, pad=token_to_idx[' '], dtype='int32', batch_first = True):
    """Casts a list of names into rnn-digestable matrix"""
    
    max_len = max_len or max(map(len, names))
    names_ix = np.zeros([len(names), max_len], dtype) + pad

    for i in range(len(names)):
        line_ix = [token_to_idx[c] for c in names[i]]
        if len(line_ix) > 20:
            print(line_ix)
        names_ix[i, :len(line_ix)] = line_ix
        
    if not batch_first: # convert [batch, time] into [time, batch]
        names_ix = np.transpose(names_ix)

    return names_ix
to_matrix(text)

array([[ 1],
       [ 1],
       [17],
       ...,
       [16],
       [ 8],
       [ 0]])

*Комментарий: т.к. у нас всего 38 различных токенов, в этот раз воспользуемся one-hot encoding'ом.*


## Попробуем генерировать слова.

Теперь наша задача - создать и обучить рекуррентную нейронную сеть, которая сможет генерировать что-то похожее на поэзию Шекспира.

Для начала воспользуемся классической RNN, аналогичной построенной на семинаре. 

In [7]:
import torch, torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [360]:
class CharRNNCell(nn.Module):
    """
    Implement the scheme above as torch module
    """
    def __init__(self, num_tokens=len(tokens), embedding_size=16, rnn_num_units=64):
        super(self.__class__,self).__init__()
        self.num_units = rnn_num_units
        
        self.embedding = nn.Embedding(num_tokens, embedding_size)
        self.rnn_update = nn.Linear(embedding_size + rnn_num_units, rnn_num_units)
        self.rnn_to_logits = nn.Linear(rnn_num_units, num_tokens)
        
    def forward(self, x, h_prev):
        """
        This method computes h_next(x, h_prev) and log P(x_next | h_next)
        We'll call it repeatedly to produce the whole sequence.
        
        :param x: batch of character ids, variable containing vector of int64
        :param h_prev: previous rnn hidden states, variable containing matrix [batch, rnn_num_units] of float32
        """
        # get vector embedding of x
        x_emb = self.embedding(x)
        
        # compute next hidden state using self.rnn_update
        # hint: use torch.cat(..., dim=...) for concatenation
        x_and_h = torch.cat((x_emb, h_prev), dim=1)
        h_next = self.rnn_update(x_and_h)
        
        h_next = F.tanh(h_next)
        
        assert h_next.size() == h_prev.size()
        
        #compute logits for next character probs
        logits = self.rnn_to_logits(h_next)
        
        return h_next, F.log_softmax(logits, -1)
    
    def initial_state(self, batch_size):
        """ return rnn state before it processes first input (aka h0) """
        return Variable(torch.zeros(batch_size, self.num_units))

In [361]:
char_rnn = CharRNNCell()

In [362]:
def rnn_loop(char_rnn, batch_ix):
    """
    Computes log P(next_character) for all time-steps in names_ix
    :param names_ix: an int32 matrix of shape [batch, time], output of to_matrix(names)
    """
    batch_size, max_length = batch_ix.size()
    hid_state = char_rnn.initial_state(batch_size)
    logprobs = []

    for x_t in batch_ix.transpose(0,1):
        hid_state, logp_next = char_rnn(x_t, hid_state)  # <-- here we call your one-step code
        logprobs.append(logp_next)
        
    return torch.stack(logprobs, dim=1)

In [363]:
num_tokens = len(tokens)
batch_ix = to_matrix(words[:5])
batch_ix = torch.tensor(batch_ix, dtype=torch.int64)

logp_seq = rnn_loop(char_rnn, batch_ix)

assert torch.max(logp_seq).data.numpy() <= 0
assert tuple(logp_seq.size()) ==  batch_ix.shape + (num_tokens,)

In [364]:
predictions_logp = logp_seq[:, :-1]
actual_next_tokens = batch_ix[:, 1:]

logp_next = torch.gather(predictions_logp, dim=2, index=actual_next_tokens[:,:,None])

loss = -logp_next.mean()

In [365]:
loss.backward()

In [366]:
for w in char_rnn.parameters():
    assert w.grad is not None and torch.max(torch.abs(w.grad)).data.numpy() != 0, \
        "Loss is not differentiable w.r.t. a weight with shape %s. Check forward method." % (w.size(),)

In [8]:
from IPython.display import clear_output
from random import sample

char_rnn = CharRNNCell()
opt = torch.optim.Adam(char_rnn.parameters())
history = []


NameError: name 'CharRNNCell' is not defined

In [368]:
MAX_LENGTH = 16
import matplotlib.pyplot as plt
%matplotlib inline
for i in range(1000):
    batch_ix = to_matrix(sample(words, 64), max_len=MAX_LENGTH)
    batch_ix = torch.tensor(batch_ix, dtype=torch.int64)
    
    logp_seq = rnn_loop(char_rnn, batch_ix)
    
    # compute loss
    #<YOUR CODE>
    predictions_logp = logp_seq[:, :-1]
    actual_next_tokens = batch_ix[:, 1:]

    loss = -torch.mean(torch.gather(predictions_logp, dim=2, index=actual_next_tokens[:,:,None]))###YOUR CODE
    
    # train with backprop
    loss.backward()
    opt.step()
    opt.zero_grad()
    #<YOUR CODE>
    
    history.append(loss.data.numpy())

assert np.mean(history[:10]) > np.mean(history[-10:]), "RNN didn't converge."

In [369]:
def generate_sample(char_rnn, seed_phrase=' ', max_length=MAX_LENGTH, temperature=1.0):
    '''
    The function generates text given a phrase of length at least SEQ_LENGTH.
    :param seed_phrase: prefix characters. The RNN is asked to continue the phrase
    :param max_length: maximum output length, including seed_phrase
    :param temperature: coefficient for sampling.  higher temperature produces more chaotic outputs,
                        smaller temperature converges to the single most likely output
    '''
    
    x_sequence = [token_to_idx[token] for token in seed_phrase]
    x_sequence = torch.tensor([x_sequence], dtype=torch.int64)
    hid_state = char_rnn.initial_state(batch_size=1)
    
    #feed the seed phrase, if any
    for i in range(len(seed_phrase) - 1):
        hid_state, _ = char_rnn(x_sequence[:, i], hid_state)
    #start generating
    for _ in range(max_length - len(seed_phrase)):
        hid_state, logp_next = char_rnn(x_sequence[:, -1], hid_state)
        p_next = F.softmax(logp_next / temperature, dim=-1).data.numpy()[0]
        # sample next token and push it back into x_sequence
        next_ix = np.random.choice(num_tokens,p=p_next)
        next_ix = torch.tensor([[next_ix]], dtype=torch.int64)
        x_sequence = torch.cat([x_sequence, next_ix], dim=1)
    return ''.join([idx_to_token[ix] for ix in x_sequence.data.numpy()[0]])

In [371]:
for _ in range(15):
    print(generate_sample(char_rnn, seed_phrase='clock'))

clocken         
clock           
clock           
clockly         
clock           
clock           
clockes         
clocksh         
clock           
clockl          
clock           
clock           
clockn          
clock           
clock           


# Работаем с текстом

In [35]:
def tokenize_doc(doc):
    # discard all punctuation
    doc = doc.translate(str.maketrans('', '', string.punctuation))
    # replace all whitespace characters with just space
    for ch in doc:
        if ch in string.whitespace :
            if ch == '\n':
                doc = doc.replace(ch,' \n')
            else:
                doc = doc.replace(ch,' ')
    # split doc into tokens by space
    tokens = []
    for token in doc.split(' '):
        if token != '':
            tokens.append(token.lower())
    # discard empty tokens and lowercase
    return tokens
words = tokenize_doc(text)

In [36]:
print(words)

['from', 'fairest', 'creatures', 'we', 'desire', 'increase', '\n', 'that', 'thereby', 'beautys', 'rose', 'might', 'never', 'die', '\n', 'but', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease', '\n', 'his', 'tender', 'heir', 'might', 'bear', 'his', 'memory', '\n', 'but', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes', '\n', 'feedst', 'thy', 'lights', 'flame', 'with', 'selfsubstantial', 'fuel', '\n', 'making', 'a', 'famine', 'where', 'abundance', 'lies', '\n', 'thy', 'self', 'thy', 'foe', 'to', 'thy', 'sweet', 'self', 'too', 'cruel', '\n', 'thou', 'that', 'art', 'now', 'the', 'worlds', 'fresh', 'ornament', '\n', 'and', 'only', 'herald', 'to', 'the', 'gaudy', 'spring', '\n', 'within', 'thine', 'own', 'bud', 'buriest', 'thy', 'content', '\n', 'and', 'tender', 'churl', 'makst', 'waste', 'in', 'niggarding', '\n', 'pity', 'the', 'world', 'or', 'else', 'this', 'glutton', 'be', '\n', 'to', 'eat', 'the', 'worlds', 'due', 'by', 'the', 'grave', 'and', 'thee', '\n', '\n', 'ii',

In [37]:
tokens = np.array(list(set(words)))
idx_to_token = {i:word for i, word in enumerate(tokens)}
token_to_idx = {word:i for i, word in enumerate(tokens)}

In [40]:
print(token_to_idx['\n'])

1434


# Енкодим текст для обучения

In [83]:
encoded_text = [token_to_idx[x] for x in words]

In [84]:
TEXT_LEN = 200

# Рандомный кусок текста заенкоженый

In [86]:
import random
import time
random.seed = 42
def random_data_encoded(batch_size=1):
    batch = []
    for _ in range(batch_size):
        index = random.randint(0,len(encoded_text)-TEXT_LEN)
        batch.append(encoded_text[index : index + TEXT_LEN])
    return batch
print(random_data_encoded())

[[350, 1201, 1862, 2005, 350, 3322, 1434, 561, 1086, 3338, 1662, 2011, 3065, 1434, 3157, 1625, 815, 900, 545, 2732, 183, 951, 1434, 1709, 350, 900, 2543, 1366, 2058, 2161, 2103, 1434, 1662, 951, 1709, 184, 595, 1662, 650, 1163, 1434, 2565, 3291, 734, 579, 1631, 1632, 3061, 129, 2601, 1434, 1434, 1213, 1434, 1434, 183, 1506, 1986, 1901, 2334, 3233, 1434, 1709, 2464, 734, 1343, 2349, 663, 379, 1434, 760, 184, 734, 3039, 1100, 645, 660, 1029, 3181, 1434, 2087, 1651, 986, 795, 1348, 2831, 610, 1434, 2565, 2706, 2925, 734, 2001, 3017, 1434, 1662, 870, 1208, 2080, 1901, 888, 1709, 2670, 1434, 1986, 2345, 2389, 2209, 1986, 420, 3326, 838, 734, 2623, 1434, 1901, 2104, 1867, 1709, 129, 2584, 1434, 1015, 1721, 1662, 1386, 1372, 1434, 734, 184, 2590, 3016, 129, 2820, 734, 1654, 2338, 785, 1434, 1373, 660, 1901, 376, 1709, 2656, 298, 1434, 3291, 2119, 2349, 2685, 2080, 1625, 1901, 1233, 2646, 1434, 951, 1946, 1901, 3276, 3291, 3296, 1560, 1434, 184, 1086, 660, 2349, 1662, 1243, 660, 1913, 376, 143

In [87]:
voc_size = len(idx_to_token)

In [88]:
class RNNLoop(nn.Module):
    
    def __init__(self, voc_size=voc_size, emb_size=TEXT_LEN, rnn_num_units=2, dropout=0.5, hidden_size=20):
        super(self.__class__, self).__init__()
        self.hidden_size = hidden_size
        self.rnn_num_units = rnn_num_units
        self.emb = nn.Embedding(voc_size, emb_size)
        self.drop = nn.Dropout(dropout)
        self.rnn = nn.GRU(emb_size, hidden_size, rnn_num_units, dropout=dropout)
        self.hid_to_logits = nn.Linear(hidden_size, voc_size)
        self.init_weights()
        
    def init_weights(self):
        self.emb.weight.data.uniform_(-0.1, 0.1)
        self.hid_to_logits.bias.data.fill_(0)
        self.hid_to_logits.weight.data.uniform_(-0.1, 0.1)
    
        
    def forward(self, x, hidden):
        x = self.emb(x)
        x = self.drop(x)
        x, hidden = self.rnn(x, hidden)
        x = self.drop(x)
        next_logits = self.hid_to_logits(x.view(x.size(0) * x.size(1), x.size(2)))
        return next_logits.view(x.size(0), x.size(1), next_logits.size(1)), hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        return Variable(weight.new(self.rnn_num_units, batch_size, self.hidden_size).zero_())
    
model = RNNLoop()
opt = torch.optim.Adam(model.parameters())
history = []

# TRAIN

In [93]:
%%time
hidden = model.init_hidden(TEXT_LEN)
for i in range(100):
    batch_ix = random_data_encoded(batch_size=200)
    batch_ix = torch.tensor(batch_ix, dtype=torch.int64)
    
    hidden = Variable(hidden.data)
    
    logp_seq, hidden = model(batch_ix, hidden)
    
    # compute loss
    predictions_logp = logp_seq[:, :-1]
    actual_next_tokens = batch_ix[:, 1:]

    loss = -torch.mean(torch.gather(predictions_logp, dim=2, index=actual_next_tokens[:,:,None]))
    
    # train with backprop
    loss.backward()
    opt.step()
    opt.zero_grad()

# assert np.mean(history[:25]) > np.mean(history[-25:]), "RNN didn't converge."

Wall time: 12min 27s


In [90]:
def generate_sample(char_rnn, seed_phrase='cool', max_length=TEXT_LEN, temperature=2.0):
    hidden = model.init_hidden(1)
    idx = token_to_idx[seed_phrase]
    x = Variable(torch.LongTensor([[idx]]).long(), volatile = True)
    words = []
    words.append(seed_phrase)
    for i in range(max_length - 1):
        logp_seq, hidden = model(x, hidden)
        word_weights = logp_seq.squeeze().data.div(temperature).exp()
        word_idx = int(torch.multinomial(word_weights, 1)[0])
        #         print(int(word_idx))
        x.data.fill_(word_idx)
        word = idx_to_token[word_idx]
        words.append(word)
    return ' '.join(words)

Сгенерируйте текст с помощью обученной сети для различных значений параметра `temperature`: `[0.1, 0.2, 0.5, 1.0, 2.0]` ("температуры" при генерации). Оцените результаты визуально, попробуйте их проинтерпретировать.

In [91]:
# Text generation with different tempearature values here
#for _ in range(50):
print(generate_sample(model, seed_phrase='since'))

first o paying and still dost to could sweet vulgar joy appetite was me where me eyes yet feeds boy i who do days rich and others be in nothing loves from beside proving art hath most from myself full thine lives 
 should even that worth far dart reignd be to dead age for woe at part from who old first hate alone i my 
 sight when still art you thinking when do why commence leave now tonguetied but this his for than to there fair hand is not the one thine have and but this on let


C:\Users\user\.conda\envs\PyTorch\lib\site-packages\ipykernel_launcher.py:4: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  after removing the cwd from sys.path.


In [27]:
model = RNNLoop()
model.load_state_dict(torch.load("model"))

In [ ]:
# Text generation with different tempearature values here
#for _ in range(50):
print(generate_sample(model, seed_phrase='first'))

Здесь можно оставить свои рассуждения касательно интерпретации результатов.

## Сохранение и загрузка модели

Сохраните обученную модель на диск, затем загрузите ее и сгенерируйте текст. Примеры доступны по [ссылке](https://pytorch.org/tutorials/beginner/saving_loading_models.html]).

In [92]:
# Saving and loading code here
torch.save(model.state_dict(), "model")

## Полезные ссылки
1. <a href='http://karpathy.github.io/2015/05/21/rnn-effectiveness/'> Статья Андрея Карпатого про RNN. </a> В качестве примеров рассматриваются задачи генерации Шекспировских текстов, Latex формул, Linux Source Code и детских имен.
2. <a href='https://github.com/karpathy/char-rnn'> Репозиторий с кодом по char-rnn </a> (тоже за авторством Андрея Карпатого)
3. Полезный репозиторий по PyTorch: [ссылка](https://github.com/spro/practical-pytorch`)